In [1]:
# Methode, um sound_in Datei zu schreiben:

import numpy as np

def write_sounding_string(ps=1013.25, zs=np.array([500,1000,1500]), Thetas=np.array([288, 289, 300]),\
                          qs=np.array([10.1, 8, 9]), us=np.array([7, 6, 8]), vs=np.array([7, 6, 8]), zmax=5000,
                         filename = "sound_in_python"):
    
    # Erste Zeile mit ps und den ersten Werten von Thetas, qs, us, vs
    text = f"  {ps:8.3f}{'':3}".zfill(8)
    for values in [Thetas[0], qs[0], us[0], vs[0]]:
        text += f"{values:7.3f}{'':3}".zfill(7)

    # Zeilen 2 bis len(zs)+1
    for z, Theta, q, u, v in zip(zs, Thetas, qs, us, vs):
        ###########
        # Ist die Null in zweiter Zeile Grund fuer dne Abbruch?
        if z==0:
            continue
        ##########3
        text += f"\n  {z:8.3f}{'':3}".zfill(8)
        for values in [Theta, q, u, v]:
            text += f"{values:7.3f}{'':3}".zfill(7)
            
        ########33
        # Abbruch in unterer Atmosphaere:
        if z>zmax:
            break
        ###########
            
    new_file = open(filename,"w")
    new_file.write(text)
    new_file.close()

    return text

In [2]:
import xarray as xr

ds = xr.open_dataset("/home/qwertz/data_js_icon_22_23_August_2018/stat_IQQ_all-times.nc")


print(ds.data_vars)
# print(ds.z_ifc.values)

string = write_sounding_string(ps=ds["pres"].isel(ncells=0).sel(time=20180822.0).values[-1]/100,\
                      zs=ds["z_mc"].isel(ncells=0).values[::-1],\
                      Thetas=ds["theta_v"].isel(ncells=0).sel(time=20180822.0).values[::-1],\
                          qs=ds["qv"].isel(ncells=0).sel(time=20180822.0).values[::-1]*1000,\
                      us=ds["u"].isel(ncells=0).sel(time=20180822.0).values[::-1],\
                      vs=ds["v"].isel(ncells=0).sel(time=20180822.0).values[::-1], zmax=5000,
                         filename = "sound_in_iconles")

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Data variables:
    clon_bnds      (ncells, vertices) float32 12B ...
    clat_bnds      (ncells, vertices) float32 12B ...
    height_bnds    (height, bnds) float64 3kB ...
    height_3_bnds  (height_3, bnds) float64 3kB ...
    qv             (time, height, ncells) float32 250kB ...
    qc             (time, height, ncells) float32 250kB ...
    qi             (time, height, ncells) float32 250kB ...
    qr             (time, height, ncells) float32 250kB ...
    qs             (time, height, ncells) float32 250kB ...
    qg             (time, height, ncells) float32 250kB ...
    qh             (time, height, ncells) float32 250kB ...
    qnc            (time, height, ncells) float32 250kB ...
    qni            (time, height, ncells) float32 250kB ...
    qnr            (time, height, ncells) float32 250kB ...
    qns            (time, height, ncells) float32 250kB ...
    qng            (time, height, ncells) float32 250kB ...
    qnh            (time, height, ncells) float32 250k

In [3]:
# Determine Vertical grid in ICON (This is only Iquique - should I talk about differences...):

new_zm = np.insert(ds["z_mc"].isel(ncells=0).values[:-50:-1], 0, 0.)
new_zt = np.insert(ds["z_ifc"].isel(ncells=0).values[:-50:-1], 0, -1.)

print(new_zm)
print(new_zt)

# Write this into z_grid file:
def write_zs_into_zgrid_file(zs, filename="zm_grid_in_example"):
    text = ""
    for z_value in zs:
        text += f"{z_value:.1f}".rjust(7)+"\n"
    
    newfile = open(filename, "w")
    newfile.write(text)
    newfile.close()
    return 0

write_zs_into_zgrid_file(new_zm)
write_zs_into_zgrid_file(new_zt, filename="zt_grid_in_ICONLES")

[   0.         59.636677   83.18278   112.13497   144.2803    178.83559
  215.38422   253.65506   293.45566   334.6415    377.0998    420.73978
  465.48672   511.2781    558.0607    605.7887    654.42224   703.92633
  754.2701    805.42596   857.3693    910.078     963.5319   1017.7129
 1072.6044   1128.1913   1184.4597   1241.397    1298.9913   1357.2319
 1416.1089   1475.613    1535.7357   1596.469    1657.8055   1719.7385
 1782.2616   1845.3688   1909.0547   1973.3142   2038.1426   2103.5352
 2169.488    2235.9976   2303.0596   2370.6714   2438.8296   2507.5315
 2576.7744   2646.556   ]
[-1.0000000e+00  4.9636677e+01  6.9636673e+01  9.6728874e+01
  1.2754107e+02  1.6101953e+02  1.9665164e+02  2.3411679e+02
  2.7319333e+02  3.1371799e+02  3.5556503e+02  3.9863455e+02
  4.4284497e+02  4.8812848e+02  5.3442773e+02  5.8169366e+02
  6.2988373e+02  6.7896075e+02  7.2889197e+02  7.7964825e+02
  8.3120367e+02  8.8353497e+02  9.3662103e+02  9.9044281e+02
  1.0449830e+03  1.1002258e+03  1.156

0

In [4]:
# Read in grid:

def read_zgrid_from_file(filename):
    zarray = []
    gridfile = open(filename, "r")
    for line in gridfile.readlines():
        zarray.append(float(line))
    gridfile.close()    
    zarray = np.array(zarray)
    return zarray

zarray = read_zgrid_from_file("zm_grid_in_example")

print(np.diff(zarray))

[59.6 23.6 28.9 32.2 34.5 36.6 38.3 39.8 41.1 42.5 43.6 44.8 45.8 46.8
 47.7 48.6 49.5 50.4 51.1 52.  52.7 53.4 54.2 54.9 55.6 56.3 56.9 57.6
 58.2 58.9 59.5 60.1 60.8 61.3 61.9 62.6 63.1 63.7 64.2 64.8 65.4 66.
 66.5 67.1 67.6 68.1 68.7 69.3 69.8]
